In [156]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [151]:
# Lets load the web pages
# https://www.immobilienscout24.de/Suche/S-T/P-1/Wohnung-Kauf/Hamburg/Hamburg/-/-1,00 
# /P-1/ -> is a page and /-1,00 -> count of rooms
req = requests.get("https://www.immobilienscout24.de/Suche/S-T/P-1/Wohnung-Kauf/Hamburg/Hamburg/-/-1,00")
content = req.content

In [152]:
# Parsing of HTML-Code
pars = BeautifulSoup(content, "html.parser")
all_divs = pars.find_all("div", {"class":"grid grid-flex gutter-horizontal-l gutter-vertical-s"})
divs_districts = pars.find_all("div", {"class":"result-list-entry__address"})

In [160]:
pattern = "\d+\.?\d+"
# get all prices as strings from page
all_prices_as_str = []
for price in all_divs:
    temp = price.find_all("dd", {"class":"font-line-xs"})[0].text[0:-2]
    temp = temp.replace('.' , '')
    all_prices_as_str.append(temp)

# convert prices into numeric
all_prices_as_float = []
for price in all_prices_as_str:
    if len(re.findall(pattern, price)) > 1:
        temp = re.findall(pattern, price)
        temp = (int(temp[0]) + int(temp[1])) / 2
        all_prices_as_float.append(temp)
    else:
        all_prices_as_float.append(int(re.findall(pattern, price)[0]))
del all_prices_as_str

# get all living spaces as strings from page
all_living_spaces_as_str = []
for space in all_divs:
    s = space.find_all("dd", {"class":"font-line-xs"})[1].text[0:-3]
    s = s.replace(',' , '.')
    all_living_spaces_as_str.append(s)

# convert all living spaces into numeric
living_spaces_as_float = []
for p in all_living_spaces_as_str:
    if len(re.findall(pattern, p)) > 1:
        temp = re.findall(pattern, p)
        temp = (float(temp[0]) + float(temp[1])) / 2
        living_spaces_as_float.append(temp)
    else:
        living_spaces_as_float.append(float(re.findall(pattern, p)[0]))
del all_living_spaces_as_str

# get all districts as strings from page
list_districts = []
for district in divs_districts:
    temp = district.find_all("div", {"class":"font-ellipsis"})[0].text
    list_districts.append(temp)
    
# convert districts into region of city
list_districts_as_str = []
for address in list_districts:
    temp = address.split(',')
    if len(temp) > 2:
        res = temp[1][1:]
    else:
        res = temp[0]
    list_districts_as_str.append(res)
del list_districts

In [154]:
# calculate the average price
price_for_one_sq_m = []
for x, y in zip(all_prices_as_float, living_spaces_as_float):
    price_for_one_sq_m.append(x / y)
del living_spaces_as_float, all_prices_as_float

In [159]:
# Group average prices by regions of city
df = {'District' : list_districts_as_str, 'Average_Price': price_for_one_sq_m}
data_frame = pd.DataFrame(data = df)
data_frame.groupby('District').mean().reset_index()

,District,Average_Price
0,Bahrenfeld,6421.194153
1,Barmbek-Süd,6642.728905
2,Billstedt,2696.969697
3,Eidelstedt,3377.358491
4,Eilbek,4125.000000
5,Eimsbüttel,5593.750000
6,Eißendorf,3450.671607
7,Eppendorf,7025.714286
8,Harvestehude,9592.379265
9,Langenhorn,2893.203883
